# Práctica 1: Aprendizaje Automático
## Parte A: Evaluación de modelos de Aprendizaje Automático
Hecho por:
- Jaime Benedí
- Miguel Sevilla

En esta parte vamos a utilizar la librería [surprise](https://surprise.readthedocs.io/en/stable/) que implementa algoritmos y otros procedimientos para desarrollar y evaluar algoritmos de sistemas de recomendación. Asegúrate de tenerla instalada antes de empezar a hacer las tareas.

Trabajaremos sobre el dataset de [MovieLens de 100K](https://grouplens.org/datasets/movielens/100k/). MovieLens es un dataset muy usado en el desarrollo y evaluación de sistemas de recomendación, y el dataset de 100k contiene 100K interacciones entre usuarios y productos, almacenando qué usuario ha puntuado qué película y con qué rating.

## Objetivos
1. Cargar MovieLens de 100k.

2. Divide el dataset en una partición simple, donde el conjunto de evaluación sea el 75% de las interacciones y el resto forme parte del conjunto de evaluación. Aquí tenéis que pasarle random_state = SEED a la función.

3. Vamos a evaluar distintos algoritmos de recomendación:
   1. Filtrado colaborativo basado en vecinos (KNNBasic en esta [librería](https://surprise.readthedocs.io/en/stable/knn_inspired.html)) tanto basado en usuarios como enproductos. Utilizaremos la métrica de similitud de Pearson.
   2. Filtrado colaborativo basado en modelos usando [factorización de matrices](https://surprise.readthedocs.io/en/stable/matrix_factorization.html), usando los algoritmos de SVD y NMF.

4. Cada uno de estos algoritmos se entrenarán con el conjunto de
entrenamiento. Aquí tenéis que pasarle random_state = SEED a cada uno de los modelos.

5. Después se obtendrán las predicciones que todos los algoritmos obtienen para el conjunto de evaluación. Muestra el resultado de 5 predicciones e interpreta los resultados.

6. Crea una tabla con los valores que se obtienen para las métricas de evaluación RMSE, precision@k, recall@k, y NDCG, k es el tamaño de la lista de recomendación y será k = 10. Surprise solo implementa RMSE. Las demás las podéis encontrar en sklearn usando precision_score, recall_score, ndcg_score.
   1. **IMPORTANTE**: solo las películas cuyo rating sea superior a 4 serán consideradas relevantes (incluyendo en la lista de películas recomendadas por el modelo).

7. Explica cada uno de los resultados obtenidos y qué significado tienen. Determina cuál podría ser el mejor método recomendador a utilizar.

## Antes de empezar...
Determina un valor para una variable SEED (el que sea). Esta variable se la vamos a pasar a los modelos para que cada vez que se ejecute el código salgan los mismos resultados. De esta forma os aseguráis de que los resultados que os salgan serán los mismos que me salgan a mí al ejecutar el código. Si no hacéis esto, vuestro análisis puede no tener ningún sentido en mi ejecución y os arriesgáis al no apto.

In [1]:
SEED = 42

In [2]:
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate

# 1 -> Cargar el dataset

In [3]:

data = Dataset.load_builtin("ml-100k")

# 2 -> Dividir el dataset:
75% conjunto de evaluación (Posible errata -> ¿Entrenamiento?)
 
 pasarle random_state = SEED a la función -> facilita la reproducibilidad de los experimentos

In [4]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25,random_state = SEED)

# 3.1 Algoritmo de recomendación: KNNBasic

In [5]:
from surprise import Dataset, KNNBasic,accuracy

# Creamos el set de entrenamiento
trainset = data.build_full_trainset()

# Construimos el algoritmo y lo entrenamos
algoKNN = KNNBasic(random_state = SEED)
algoKNN.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [6]:
# Generar predicciones en el conjunto de prueba
predictionsKNN = algoKNN.test(testset)

# Calcular el RMSE para evaluar el error del modelo
rmseKNN  = accuracy.rmse(predictionsKNN)


RMSE: 0.7789


RMSE (Root Mean Squared Error) -> mide la precisión del modelo, cuanto menor sea esta mayor sera la capacidad del modelo para hacer sus predicciones. 0.7789 es un valor aceptable , sin embargo para saber cual es el mas adecuado para este dataset vamos a calcular primero el SVD y el NMF para comparar.

# 3.2 Algoritmo de recomendación: SVD y NMF
 
## SVD (Single Value Decomposition)
 Tecnica matematica que permite descomponer una matriz en tres matrices A = u∑V^T
 - U -> Matriz vectores singulares izquierdos (direcciones principales en el espacio original de los datos)
 - ∑ -> Matriz diagonal con los valores singulares ("fuerza" de cada direccion)
 - V^T -> Traspuesta de la matriz de vectores singulares derechos , dan una base para el espacio de las variables

 Esta composicion permite entender la estructura de la información contenida en A, al conservar solo los componentes más importantes (mayor valor singular), se reduce la dimensionalidad de los datos y se elimina el ruido.

 En sistemas de recomendación, la matriz A representa las calificaciones de usuarios a diferentes ítems. Debido a que esta matriz es muy dispersa (muchos datos faltantes), no es posible aplicar directamente la SVD clásica. En su lugar, se aproxima la matriz mediante una factorización en componentes latentes, buscando un ajuste óptimo que minimice el error sobre las calificaciones conocidas.
 
 Al ejecutar algoSVD.fit(trainset), el algoritmo estima μ,bu,bi,pu y qi:
- μ -> calificación pormedio global
- bu -> sesgo del usuario u 
- bi -> sesgo del item
- pu -> vector de factores latentes del usuario (resume las características o preferencias ocultas del usuario)
- qi -> vector de factores latentes del item i (Descripción del producto a través de un vector de N factores                   latentes)

 La función algoSVD.test(testset) utiliza estos paremtros para predecir las calificaciones en el conjunto de prueba

 Y por ultimo RMSE mide la precisión de las predicciones (RMSE -> medida que indica como de lejos están las predicciones de los valores reales , por lo tanto a menor valor mejor modelo)

 ## NMF (Non-Negative Matriz Factorization)
 Tecnica de fcatorización de matrices que descompone una matriz A en dos matrices no negativas, A ≈ W . H
 
 - W es la matriz de factores latetes de los usuarios (fila -> usuario , columna -> caracteristica latente)
 - H es la matriz de factores latentes de los items (columna -> item, fila -> carcaterística latente

 En este ejemplo como estamos trabajando con peliculas, los factores latentes podrían representar características como   accíon, suspense o romántico, y cada película tendría una cantidad de estas, siempre con numeros positivos, evitando   que se cancelen entre si.
  

In [7]:

from surprise import accuracy

algoSVD = SVD(random_state = SEED)

# Entrenar el algoritmo con el trainset y predecir los ratios para el test
algoSVD.fit(trainset)
predictionsSVD = algoSVD.test(testset)

rmseSVD = accuracy.rmse(predictionsSVD)

RMSE: 0.6757


In [8]:
from surprise import NMF

# Crear una instancia del modelo NMF con la misma semilla para comparar
algoNMF = NMF(random_state=SEED)

# Entrenar el modelo en el conjunto de entrenamiento
algoNMF.fit(trainset)

# Predecir las calificaciones para el conjunto de prueba
predictionsNMF = algoNMF.test(testset)

# Calcular el RMSE para evaluar el error del modelo
rmseNMF = accuracy.rmse(predictionsNMF)


RMSE: 0.8233


El RMSE empleando este algoritmo (NMF) y el algoritmo KNN son mayores que el de SVD por lo tanto SVD tiene un mejor desempeño en este caso y el algoritmo más recomendado

# 5. Muestra el resultado de 5 predicciones

In [9]:
print("Predicciones KNN")
for pred in predictionsKNN[:5]:  # Tomar las primeras 5
    uid, iid, true_r, est, _ = pred
    print(f"Usuario {uid} - Película {iid} | Real: {true_r} | Prediccion: {est:.2f}")

print("\nPredicciones SVD")
for pred in predictionsSVD[:5]:  # Tomar las primeras 5
    uid, iid, true_r, est, _ = pred
    print(f"Usuario {uid} - Película {iid} | Real: {true_r} | Prediccion: {est:.2f}")

print("\nPredicciones NMF")
for pred in predictionsNMF[:5]:  # Tomar las primeras 5
    uid, iid, true_r, est, _ = pred
    print(f"Usuario {uid} - Película {iid} | Real: {true_r} | Prediccion: {est:.2f}")

Predicciones KNN
Usuario 391 - Película 591 | Real: 4.0 | Prediccion: 3.63
Usuario 181 - Película 1291 | Real: 1.0 | Prediccion: 1.94
Usuario 637 - Película 268 | Real: 2.0 | Prediccion: 2.82
Usuario 332 - Película 451 | Real: 5.0 | Prediccion: 3.92
Usuario 271 - Película 204 | Real: 4.0 | Prediccion: 3.95

Predicciones SVD
Usuario 391 - Película 591 | Real: 4.0 | Prediccion: 3.54
Usuario 181 - Película 1291 | Real: 1.0 | Prediccion: 1.00
Usuario 637 - Película 268 | Real: 2.0 | Prediccion: 2.42
Usuario 332 - Película 451 | Real: 5.0 | Prediccion: 4.45
Usuario 271 - Película 204 | Real: 4.0 | Prediccion: 3.74

Predicciones NMF
Usuario 391 - Película 591 | Real: 4.0 | Prediccion: 3.61
Usuario 181 - Película 1291 | Real: 1.0 | Prediccion: 1.00
Usuario 637 - Película 268 | Real: 2.0 | Prediccion: 2.46
Usuario 332 - Película 451 | Real: 5.0 | Prediccion: 4.38
Usuario 271 - Película 204 | Real: 4.0 | Prediccion: 3.81


KNN tiende a alejarse más de los valores reales como por ejemplo en el caso 2 donde el rating real es 1 y sin embargo predice 1,94 así como en el caso 4 predice 3,95 cuando el valor real es 5. Esto quiere decir que filtrar en base a vecinos (KNN) no es tan preciso en este contexto.
Por otro lado SVD y NMF son más precisos , destacando el SVD que como se predecía con su valor de RMSE (el más bajo de los 3) es el que más se aproxima en la mayoría de casos.


# 6: Tabla con los valores de RMSE, precision@k, recall@k, y NDCG,

 k es el tamaño de la lista de recomendación y será k = 10. Sklearn para precision_score, recall_score, ndcg_score.
 
 Rating superior o igual a 4

In [18]:
# sklearn.metrics, necesita convertir los valores en listas numpy.
from sklearn.metrics import precision_score, recall_score, ndcg_score
import pandas as pd

#1-> Valores a 0 o 1 , siendo 1 relevante y 0 no relevante, para > 4 entonces relevante
def get_binary_relevance(predictions):
    #Valores reales a binarios
    y_true = [1 if true_r >= 4 else 0 for (_, _, true_r, _, _) in predictions]
    #Valores predicciones a binarios
    y_pred = [1 if est >=4 else 0 for (_, _, _, est, _) in predictions]
    return y_true, y_pred
    
y_true_knn, y_pred_knn = get_binary_relevance(predictionsKNN)
y_true_svd, y_pred_svd = get_binary_relevance(predictionsSVD)
y_true_nmf, y_pred_nmf = get_binary_relevance(predictionsNMF)

In [19]:
#2-> Función para calcular precisión, recall y NDCG
def calcula_metricas(y_true, y_pred, k=10):
    #Proporción de predicciones positivas correctas
    precision = precision_score(y_true, y_pred, zero_division=0)
    #Proporción de elementos relevantes que han sido identificados                           
    recall = recall_score(y_true, y_pred, zero_division=0)
    #Evaluar la calidad del ranking (k limita el calculo a el top k elementos
    ndcg = ndcg_score([y_true], [y_pred], k=k)  # NDCG@K
    return precision, recall, ndcg
    
# Calcular métricas para cada modelo
precisionKNN, recallKNN, ndcgKNN = calcula_metricas(y_true_knn, y_pred_knn)
precisionSVD, recallSVD, ndcgSVD = calcula_metricas(y_true_svd, y_pred_svd)
precisionNMF, recallNMF, ndcgNMF = calcula_metricas(y_true_nmf, y_pred_nmf)

In [20]:
# Crear la tabla con todos los resultados
results = pd.DataFrame({
    "Modelo": ["KNN", "SVD", "NMF"],
    "RMSE": [rmseKNN, rmseSVD, rmseNMF],  # Ya los tienes calculados
    "Precision@10": [precisionKNN, precisionSVD, precisionNMF],
    "Recall@10": [recallKNN, recallSVD, recallNMF],
    "NDCG@10": [ndcgKNN, ndcgSVD, ndcgNMF]
})

print(results)

  Modelo      RMSE  Precision@10  Recall@10   NDCG@10
0    KNN  0.778858      0.944406   0.486473  0.944406
1    SVD  0.675727      0.961173   0.488056  0.961173
2    NMF  0.823313      0.896519   0.409555  0.896519


Menor RMSE -> Mejor -> SVD

Precision(% peliculas realmente importantes) -> Mayor Precison -> Mejor -> SVD

Recall(% peliculas relevantes recomendadas) -> Mayor Recall -> Mejor -> SVD

NDCG(peliculas mas relevantes en primeras posiciones) -> Mayor NDCG -> Mejor -> SVD

El mejor modelo claramente es SVD